In [1]:
import pandas as pd

In [2]:
AI = pd.read_csv('AI_Resume_Screening.csv')

In [3]:
AIrD = pd.DataFrame(AI)

# dont need resume id or name
AIrD = AIrD.drop(labels=['Resume_ID', 'Name'], axis=1)

# ----- education
AIrD['Education'] = AIrD['Education'].astype(str).str.strip().str.lower()

education_mapping = {
    'b.sc': 1,
    'b.tech': 2,
    'btech': 2,        # just in case
    'mba': 3,
    'm.tech': 4,
    'phd': 5
}

AIrD['Education'] = AIrD['Education'].map(education_mapping)

AIrD = AIrD.dropna(subset=['Education'])

# ----- recruiter decision: 1 hired 0 rejected
AIrD['Recruiter Decision'] = AIrD['Recruiter Decision'].astype(str).str.strip().str.lower()

AIrD['Recruiter Decision'] = AIrD['Recruiter Decision'].map({
    'hire': 1,
    'reject': 0
})

AIrD = AIrD.dropna(subset=['Recruiter Decision'])

AIrD = AIrD.reset_index(drop=True)

AIrD

,Skills,Experience (Years),Education,Certifications,Job Role,Recruiter Decision,Salary Expectation ($),Projects Count,AI Score (0-100)
0,"TensorFlow, NLP, Pytorch",10,1,NaN,AI Researcher,1,104895,8,100
1,"Deep Learning, Machine Learning, Python, SQL",10,3,Google ML,Data Scientist,1,113002,1,100
2,"Ethical Hacking, Cybersecurity, Linux",1,3,Deep Learning Specialization,Cybersecurity Analyst,1,71766,7,70
3,"Python, Pytorch, TensorFlow",7,2,AWS Certified,AI Researcher,1,46848,0,95
4,"SQL, React, Java",4,5,NaN,Software Engineer,1,87441,9,100
...,...,...,...,...,...,...,...,...,...
995,"Cybersecurity, Linux, Ethical Hacking",0,1,NaN,Cybersecurity Analyst,0,114364,9,60
996,"Deep Learning, Machine Learning",0,3,Deep Learning Specialization,Data Scientist,0,103294,5,45
997,"TensorFlow, NLP",0,2,Google ML,AI Researcher,1,113855,9,65
998,"Linux, Networking, Cybersecurity, Ethical Hacking",8,5,AWS Certified,Cybersecurity Analyst,1,83146,10,100


In [4]:
AIrD.to_csv('cleaned_AI_resume_data.csv', index=False)

In [5]:
#logistic Regression, Naive Bayes, Random Forest
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
#from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

%pip install graphviz
import graphviz

Note: you may need to restart the kernel to use updated packages.


In [6]:
# LOGISTIC REGRESSION - looking at pair plot, decided to use features: experince, projects, and ai score
X = AIrD[['AI Score (0-100)', 'Experience (Years)', 'Education']]
y = AIrD['Recruiter Decision']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

confusion_matrix(y_test, y_pred)

array([[ 46,   0],
       [  0, 154]], dtype=int64)

In [7]:
y.value_counts()

Recruiter Decision
1    812
0    188
Name: count, dtype: int64

In [8]:
# DECISION TREE -
X = AIrD.drop(labels= ['Recruiter Decision', "Job Role"], axis=1)
y = AIrD['Recruiter Decision']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

clf = tree.DecisionTreeClassifier(criterion='gini')
X_train_encoded = pd.get_dummies(X_train, columns=['Certifications', 'Skills'])
clf.fit(X_train_encoded, y_train)


graph_data = tree.export_graphviz(clf, out_file=None, feature_names=list(X_train_encoded.columns),
                                  class_names=list(set(y_train)), filled=True, rounded=True)
graph = graphviz.Source(graph_data, )
graph

TypeError: can only concatenate str (not "numpy.int32") to str